In [100]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference,LineChart
from openpyxl.chart.label import DataLabelList
from openpyxl.styles import Alignment  
from openpyxl.styles import PatternFill, Font
from datetime import datetime

In [79]:
new_file_name = "dummy_data"
df = pd.read_excel(f"{new_file_name}_raw.xlsx")
df.head()

,plugin_id,plugin_family,severity,asset_group,patch_publication_date,first_observed_date
0,5449,Cloud,Critical,Production,06/03/2025 19:53:39,18/01/2025 16:07:14
1,9323,Operating Systems,Critical,UAT,19/03/2025 20:59:05,23/05/2025 07:06:50
2,8232,Network Devices,Medium,UAT,15/07/2025 11:38:30,03/06/2025 21:33:03
3,5523,Databases,Critical,UAT,14/04/2025 08:03:21,12/02/2025 05:18:16
4,6594,Databases,Medium,DR,19/08/2025 07:44:57,14/03/2025 08:38:46


In [80]:
df['patch_publication_date'] = pd.to_datetime(df['patch_publication_date'], format = "%d/%m/%Y %H:%M:%S")
df['first_observed_date'] = pd.to_datetime(df['first_observed_date'], format="%d/%m/%Y %H:%M:%S")

df.head()

,plugin_id,plugin_family,severity,asset_group,patch_publication_date,first_observed_date
0,5449,Cloud,Critical,Production,2025-03-06 19:53:39,2025-01-18 16:07:14
1,9323,Operating Systems,Critical,UAT,2025-03-19 20:59:05,2025-05-23 07:06:50
2,8232,Network Devices,Medium,UAT,2025-07-15 11:38:30,2025-06-03 21:33:03
3,5523,Databases,Critical,UAT,2025-04-14 08:03:21,2025-02-12 05:18:16
4,6594,Databases,Medium,DR,2025-08-19 07:44:57,2025-03-14 08:38:46


In [81]:
today = datetime.today()
df['comparison_date'] = pd.to_datetime(today, format="%d/%m/%Y %H:%M:%S")
for idx, row in df.iterrows():
    if pd.isnull(row['patch_publication_date']):
        df.at[idx, "difference"] = (row['comparison_date'] - row['first_observed_date']).days
    else:
        df.at[idx, "difference"] = (row['comparison_date'] - row['patch_publication_date']).days

severity_thresholds = {"Critical": 60, "High": 60, "Medium": 90, "Low": 90}

df["overdue"] = df.apply(
    lambda row: "Y" if row["difference"] > severity_thresholds[row["severity"]] else "N",
    axis=1
)

df.head()

,plugin_id,plugin_family,severity,asset_group,patch_publication_date,first_observed_date,comparison_date,difference,overdue
0,5449,Cloud,Critical,Production,2025-03-06 19:53:39,2025-01-18 16:07:14,2025-09-10 19:36:21.591700,187.0,Y
1,9323,Operating Systems,Critical,UAT,2025-03-19 20:59:05,2025-05-23 07:06:50,2025-09-10 19:36:21.591700,174.0,Y
2,8232,Network Devices,Medium,UAT,2025-07-15 11:38:30,2025-06-03 21:33:03,2025-09-10 19:36:21.591700,57.0,N
3,5523,Databases,Critical,UAT,2025-04-14 08:03:21,2025-02-12 05:18:16,2025-09-10 19:36:21.591700,149.0,Y
4,6594,Databases,Medium,DR,2025-08-19 07:44:57,2025-03-14 08:38:46,2025-09-10 19:36:21.591700,22.0,N


In [83]:
with pd.ExcelWriter(f"{new_file_name}.xlsx", engine='openpyxl', mode='a', if_sheet_exists="overlay") as writer:
    df.to_excel(writer, sheet_name='Working_Sheet',index = False)

In [84]:
family_vul = df['plugin_family'].value_counts().reset_index()
family_vul.columns = ['family', 'count']
family_vul_rows = family_vul.shape[0]
family_vul_columns = family_vul.shape[1]
family_vul

,family,count
0,Cloud,33
1,Network Devices,28
2,Operating Systems,26
3,Databases,25
4,General,23
5,Middleware,22
6,Applications,22
7,Web Servers,21


In [85]:
severity_vul = df['severity'].value_counts().reset_index()
severity_vul.columns = ['severity', 'count']
severity_vul_rows = severity_vul.shape[0]
severity_vul_columns = severity_vul.shape[1]
severity_vul

,severity,count
0,Medium,58
1,Critical,51
2,Low,46
3,High,45


In [86]:
asset_group_vul = df['asset_group'].value_counts().reset_index()
asset_group_vul.columns = ['asset_group', 'count']
asset_group_vul_rows = asset_group_vul.shape[0]
asset_group_vul_columns = asset_group_vul.shape[1]
asset_group_vul

,asset_group,count
0,UAT,51
1,Production,40
2,DR,39
3,Test,36
4,Development,34


In [87]:
overdue_vul = df['overdue'].value_counts().reset_index()
overdue_vul.columns = ['overdue', 'count']
overdue_vul_rows = overdue_vul.shape[0]
overdue_vul_columns = overdue_vul.shape[1]
overdue_vul

,overdue,count
0,Y,138
1,N,62


In [88]:
pivot_table_1 = pd.pivot_table(df, index="plugin_family", columns="severity", values="plugin_id", aggfunc="count", fill_value=0).stack().reset_index()
pivot_table_1.columns = ['plugin_family', 'severity', 'count']
pivot_table_1_rows = pivot_table_1.shape[0]
pivot_table_1_columns = pivot_table_1.shape[1]
pivot_table_1

,plugin_family,severity,count
0,Applications,Critical,6
1,Applications,High,6
2,Applications,Low,3
3,Applications,Medium,7
4,Cloud,Critical,10
5,Cloud,High,6
6,Cloud,Low,12
7,Cloud,Medium,5
8,Databases,Critical,6
9,Databases,High,5


In [89]:
pivot_table_2 = pd.pivot_table(df, index="asset_group", columns="severity", values="plugin_id", aggfunc="count", fill_value=0).stack().reset_index()
pivot_table_2.columns = ['asset_group', 'severity', 'count']
pivot_table_2_rows = pivot_table_2.shape[0]
pivot_table_2_columns = pivot_table_2.shape[1]
pivot_table_2

,asset_group,severity,count
0,DR,Critical,15
1,DR,High,9
2,DR,Low,7
3,DR,Medium,8
4,Development,Critical,6
5,Development,High,11
6,Development,Low,8
7,Development,Medium,9
8,Production,Critical,9
9,Production,High,7


In [90]:
pivot_table_3 = pd.pivot_table(df, index="overdue", columns="severity", values="plugin_id", aggfunc="count", fill_value=0).stack().reset_index()
pivot_table_3.columns = ['overdue', 'severity', 'count']
pivot_table_3_rows = pivot_table_3.shape[0]
pivot_table_3_columns = pivot_table_3.shape[1]
pivot_table_3

,overdue,severity,count
0,N,Critical,11
1,N,High,13
2,N,Low,14
3,N,Medium,24
4,Y,Critical,40
5,Y,High,32
6,Y,Low,32
7,Y,Medium,34


In [91]:
pivot_table_4 = pd.pivot_table(df, index=["asset_group", "plugin_family"], columns="severity", values="plugin_id", aggfunc="count", fill_value=0).stack().reset_index()
pivot_table_4.columns = ['asset_group', 'plugin_family', 'severity', 'count']
pivot_table_4_rows = pivot_table_4.shape[0]
pivot_table_4_columns = pivot_table_4.shape[1]
pivot_table_4

,asset_group,plugin_family,severity,count
0,DR,Applications,Critical,1
1,DR,Applications,High,1
2,DR,Applications,Low,0
3,DR,Applications,Medium,1
4,DR,Cloud,Critical,4
...,...,...,...,...
155,UAT,Operating Systems,Medium,3
156,UAT,Web Servers,Critical,1
157,UAT,Web Servers,High,0
158,UAT,Web Servers,Low,1


In [94]:
pivot_table_5 = pd.pivot_table(df, index=["asset_group", "plugin_family", "overdue"], columns="severity", values="plugin_id", aggfunc="count", fill_value=0).stack().reset_index()
pivot_table_5.columns = ['asset_group', 'plugin_family', 'overdue', 'severity', 'count']
pivot_table_5_rows = pivot_table_5.shape[0]
pivot_table_5_columns = pivot_table_5.shape[1]
pivot_table_5

,asset_group,plugin_family,overdue,severity,count
0,DR,Applications,Y,Critical,1
1,DR,Applications,Y,High,1
2,DR,Applications,Y,Low,0
3,DR,Applications,Y,Medium,1
4,DR,Cloud,N,Critical,1
...,...,...,...,...,...
283,UAT,Web Servers,N,Medium,1
284,UAT,Web Servers,Y,Critical,1
285,UAT,Web Servers,Y,High,0
286,UAT,Web Servers,Y,Low,1


In [95]:
with pd.ExcelWriter(f"{new_file_name}.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    sheet = "Summary"
    family_vul.to_excel(writer, sheet_name=sheet, startrow=0, startcol=0, index=False)
    new_row = family_vul_rows + 2
    severity_vul.to_excel(writer, sheet_name=sheet, startrow=new_row, startcol=0, index=False)
    new_row = new_row + severity_vul_rows + 2
    asset_group_vul.to_excel(writer, sheet_name=sheet, startrow=new_row, startcol=0, index=False)
    new_row = new_row + asset_group_vul_rows + 2
    overdue_vul.to_excel(writer, sheet_name=sheet, startrow=new_row, startcol=0, index=False)
    new_col = family_vul_columns + 2
    pivot_table_1.to_excel(writer, sheet_name=sheet, startrow=0, startcol=new_col, index=False)
    new_col = new_col + pivot_table_1_columns + 2
    pivot_table_2.to_excel(writer, sheet_name=sheet, startrow=0, startcol=new_col, index=False)
    new_col = new_col + pivot_table_2_columns + 2
    pivot_table_3.to_excel(writer, sheet_name=sheet, startrow=0, startcol=new_col, index=False)
    new_col = new_col + pivot_table_3_columns + 2
    pivot_table_4.to_excel(writer, sheet_name=sheet, startrow=0, startcol=new_col, index=False)
    new_col = new_col + pivot_table_4_columns + 2
    pivot_table_5.to_excel(writer, sheet_name=sheet, startrow=0, startcol=new_col, index=False)

    ws = writer.sheets[sheet]

    header_row = 1
    first_data_row = header_row+1
    last_data_row = ws.max_row
    start_col = family_vul_columns + 2 + 1
    last_col = start_col + pivot_table_1_columns + 2 + pivot_table_2_columns + 2 + pivot_table_3_columns + 2 + pivot_table_4_columns + 2 + pivot_table_5_columns - 1

    ws.auto_filter.ref = f"{get_column_letter(start_col)}{header_row}:{get_column_letter(last_col)}{last_data_row}"
    

In [140]:
wb = load_workbook(f"{new_file_name}.xlsx")
sheet = wb['Summary']

chart1 = BarChart()
chart1.type = "col"
chart1.style = 10
chart1.title = "Vulnerabilities by Family"
chart1.title.overlay = False
chart1.y_axis.title = 'Count'
chart1.y_axis.overlay = False
chart1.x_axis.title = 'Family'
chart1.x_axis.overlay = False

data1 = Reference(sheet, min_col=2, min_row=1, max_row=family_vul_rows+1, max_col=family_vul_columns)
cats1 = Reference(sheet, min_col=1, min_row=2, max_row=family_vul_rows+1)
chart1.add_data(data1, titles_from_data=True)
chart1.dataLabels = DataLabelList() 
chart1.dataLabels.showVal = True
chart1.dataLabels.showSerName = False
chart1.dataLabels.showCatName = False
chart1.dataLabels.showLeaderLines = False
chart1.set_categories(cats1)
chart1.shape = 4
chart1.width = 25
chart1.height = 10

In [141]:
blank_df = pd.DataFrame()

with pd.ExcelWriter(f"{new_file_name}.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    sheet = 'Overview'
    blank_df.to_excel(writer, sheet_name=sheet)

wb = load_workbook(f"{new_file_name}.xlsx")
sheet = wb['Overview']
sheet.add_chart(chart1,'A5')
wb.save(f"{new_file_name}.xlsx")

In [131]:
chart1.y_axis

<openpyxl.chart.axis.NumericAxis object>
Parameters:
axId=100, scaling=<openpyxl.chart.axis.Scaling object>
Parameters:
logBase=None, orientation='minMax', max=None, min=None, delete=None, axPos='l', majorGridlines=<openpyxl.chart.axis.ChartLines object>
Parameters:
spPr=None, minorGridlines=None, title=<openpyxl.chart.title.Title object>
Parameters:
tx=<openpyxl.chart.text.Text object>
Parameters:
strRef=None, rich=<openpyxl.chart.text.RichText object>
Parameters:
bodyPr=<openpyxl.drawing.text.RichTextProperties object>
Parameters:
rot=None, spcFirstLastPara=None, vertOverflow=None, horzOverflow=None, vert=None, wrap=None, lIns=None, tIns=None, rIns=None, bIns=None, numCol=None, spcCol=None, rtlCol=None, fromWordArt=None, anchor=None, anchorCtr=None, forceAA=None, upright=None, compatLnSpc=None, prstTxWarp=None, scene3d=None, noAutofit=False, normAutofit=False, spAutoFit=False, lstStyle=None, p=[<openpyxl.drawing.text.Paragraph object>
Parameters:
pPr=<openpyxl.drawing.text.ParagraphP